In [125]:
from config import *
import pandas as pd
import numpy as np
import os

In [126]:
train = pd.read_csv(os.path.join(DATA_DIR, train))
test = pd.read_csv(os.path.join(DATA_DIR, test))

In [6]:
#print train.shape, test.shape
#print train.head()
#print test.head()
print train.head()
#rint test.columns

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [ ]:
categoryVar = ['Survived', 'Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked']
contVar = ['Age', 'SibSp', 'Parch']

In [22]:
train['Sex'] = train['Sex'].astype('category')
train['Survived'] = train['Survived'].astype('category')
pd.crosstab(train['Sex'], train['Survived'], normalize='index') 

Survived,0,1
Sex,,
female,0.257962,0.742038
male,0.811092,0.188908


In [29]:
train['child'] = np.where(train['Age'] < 18, 1, 0)

In [36]:
#train.groupby(['Sex', 'child', 'Survived']).size()
#pd.crosstab([train['Sex'], train['child']] ,train['Survived'], normalize='index') 

train['Pclass'] = train['Pclass'].astype('category')

train['FareBin'] = np.where(test['Fare']=='female' and test['Child'] = 1, 1, 0)

train['Survived'] = train['Survived'].astype('category')
pd.crosstab(train['Sex'], train['Survived'], normalize='index') 

Survived             0         1
Sex    child                    
female 0      0.247104  0.752896
       1      0.309091  0.690909
male   0      0.834297  0.165703
       1      0.603448  0.396552

In [38]:
def check_secure(row):
    if row['Fare'] > 30:
        return '30+'
    elif row['Fare'] <= 30 and row['Fare'] >= 20:
        return '20-30'
    elif row['Fare'] <= 20 and row['Fare'] >= 10:
        return '10-20'
    else:
        return '<10'

train['FareBin'] = train.apply(check_secure, axis=1)
train.head()

pd.crosstab([train['Sex'], train['FareBin']], train['Survived'], normalize='index') 

Survived               0         1
Sex    FareBin                    
female 10-20    0.269231  0.730769
       20-30    0.305085  0.694915
       30+      0.141593  0.858407
       <10      0.406250  0.593750
male   10-20    0.811881  0.188119
       20-30    0.734940  0.265060
       30+      0.677686  0.322314
       <10      0.893382  0.106618

In [73]:
#test['Survived'] = 0
# test.head()
# test['Sex'] = test['Sex'].astype('category')
# test['Pclass'] = test['Pclass'].astype('category')
test['Child'] = np.where(test['Age'] < 18, 1, 0)
# #test['Survived'] = np.where(test['Sex']=='female', 1, 0)

def check_secure1(row):
    if row['Sex']=='female' and row['Pclass'] == 3 and row['Fare']>=2:
        return 1
    elif row['Sex']=='female':
        return 1
    elif row['Child']==1:
        return 1
    else:
        return 0

test['Survived'] = test.apply(check_secure1, axis=1)

test.head()
# test1 = test.ix[:,['PassengerId', 'Survived']]
# print test1.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Child
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,0


In [74]:
test1 = test.ix[:,['PassengerId', 'Survived']]
test1.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [75]:
test1.to_csv(os.path.join(DATA_DIR, 'gender_final1.csv'), index=False)

In [127]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [129]:
train['Pclass'] = train['Pclass'].astype('category')
train['Sex'] = train['Sex'].astype('category')
train['Embarked'] = train['Embarked'].astype('category')
train['Survived'] = train['Survived'].astype('category')


from sklearn.preprocessing import Imputer, LabelEncoder
import pickle, os
def get_model(fileName):
    output = open(os.path.join(DATA_DIR,fileName), 'rb')
    le = pickle.load(output)
    output.close()
    return le

def set_model(clf, fileName):
    output = open(os.path.join(DATA_DIR,fileName), 'wb')
    pickle.dump(clf, output)
    output.close()

#categorical_features = ['Pclass', 'Sex', 'Embarked', 'Survived']
def preprocess(df, categorical_features, training=True):
    for col in categorical_features:
        if training:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            set_model(le, 'LabelEncoder.pkl')
            df[col] = df[col].fillna(df[col].mode())
        else:
            le  = get_model('LabelEncoder.pkl')
            df[col] = le.fit_transform(df[col].astype(str))
    return df
train= preprocess(train, ['Pclass', 'Sex', 'Embarked', 'Survived'])
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Parch'] = train['Parch'].fillna(train['Parch'].mean())
train['Fare'] = train['Fare'].fillna(train['Fare'].mean())

data = train.ix[:,[ 'Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked']]
target = train.ix[:,['Survived']]
print data.head()
print target.head()

   Pclass  Sex   Age  Parch     Fare  Embarked
0       2    1  22.0      0   7.2500         2
1       0    0  38.0      0  71.2833         0
2       2    0  26.0      0   7.9250         2
3       0    0  35.0      0  53.1000         2
4       2    1  35.0      0   8.0500         2
   Survived
0         0
1         1
2         1
3         1
4         0


In [116]:
data['Embarked'].value_counts()

2    644
0    168
1     77
3      2
Name: Embarked, dtype: int64

In [153]:
clf_gini = DecisionTreeClassifier(criterion = "entropy")
clf_gini.fit(data, target)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [154]:
test['Pclass'] = test['Pclass'].astype('category')
test['Sex'] = test['Sex'].astype('category')
test['Embarked'] = test['Embarked'].astype('category')

test = preprocess(test, ['Pclass', 'Sex', 'Embarked'], False)
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Parch'] = test['Parch'].fillna(test['Parch'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())
data1 = test.ix[:,[ 'Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked']]
print test.shape
pred = clf_gini.predict(data1)

(418, 12)


In [155]:
len(pred)

418

In [156]:
ids = test['PassengerId']
len(np.array(ids))

418

In [157]:
test['Survived'] = pred.tolist()

In [158]:
test1 = test.ix[:,['PassengerId', 'Survived']]
test1.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [159]:
test1.to_csv(os.path.join(DATA_DIR, 'gender_finalEntropy.csv'), index=False)